In [36]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import date

import acq
import env

# Exploring
import scipy.stats as stats
from sklearn.model_selection import train_test_split

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df = acq.get_zillow_data()
df.head(2)

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.00,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,261.00,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None


In [3]:
df = df.sample(5000)
df.head(2)

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
1113,11871192,NaN,NaN,266.00,2.00,NaN,NaN,1.00,2361444,NaN,...,1115,0.01,2017-01-05,Central,None,None,Central,Condominium,None,None
75345,14442954,NaN,NaN,261.00,NaN,NaN,NaN,NaN,877590,NaN,...,75379,0.03,2017-09-11,None,None,None,None,Single Family Residential,None,None


In [4]:
df.shape

(5000, 69)

In [5]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
#function that will drop rows or columns based on the percent of values that are missing:\
#handle_missing_values(df, prop_required_column, prop_required_row
    threshold = int(round(prop_required_column*len(df.index),0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [6]:
df = handle_missing_values(df, prop_required_column = .5, prop_required_row = .70)
df.head()

,parcelid,propertylandusetypeid,heatingorsystemtypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
1113,11871192,266.00,2.00,2361444,1.00,1.00,8.00,1.00,795.00,795.00,...,"282,954.00","2,016.00","159,196.00","3,478.40","60,372,013,022,000.00",1115,0.01,2017-01-05,Central,Condominium
75345,14442954,261.00,NaN,877590,2.50,3.00,NaN,2.50,"1,825.00","1,825.00",...,"845,908.00","2,016.00","637,793.00","9,695.64","60,590,423,392,002.00",75379,0.03,2017-09-11,None,Single Family Residential
10529,14667466,266.00,NaN,788639,1.00,1.00,NaN,1.00,678.00,678.00,...,"294,422.00","2,016.00","218,957.00","3,732.56","60,590,756,054,012.00",10538,-0.04,2017-02-16,None,Condominium
26337,11488432,261.00,2.00,1077439,3.00,4.00,9.00,3.00,"2,891.00","2,891.00",...,"1,293,500.00","2,016.00","969,700.00","15,250.74","60,376,205,222,002.00",26350,0.01,2017-04-14,Central,Single Family Residential
76037,12682581,266.00,2.00,192882,4.00,4.00,11.00,4.00,"2,520.00","2,520.00",...,"870,000.00","2,016.00","348,400.00","9,890.20","60,376,513,021,011.00",76071,0.06,2017-09-13,Central,Condominium


In [7]:
df.shape

(4998, 35)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 1113 to 41622
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      4998 non-null   int64  
 1   propertylandusetypeid         4998 non-null   float64
 2   heatingorsystemtypeid         3162 non-null   float64
 3   id                            4998 non-null   int64  
 4   bathroomcnt                   4998 non-null   float64
 5   bedroomcnt                    4998 non-null   float64
 6   buildingqualitytypeid         3181 non-null   float64
 7   calculatedbathnbr             4961 non-null   float64
 8   calculatedfinishedsquarefeet  4983 non-null   float64
 9   finishedsquarefeet12          4758 non-null   float64
 10  fips                          4998 non-null   float64
 11  fullbathcnt                   4961 non-null   float64
 12  latitude                      4998 non-null   float64
 13 

In [9]:
 df['heatingorsystemtypeid'] = df.heatingorsystemtypeid.fillna(value = df['heatingorsystemtypeid'].mean())

In [10]:
 df['buildingqualitytypeid'] = df.buildingqualitytypeid.fillna(value = df['buildingqualitytypeid'].mean())

In [11]:
 df['calculatedbathnbr'] = df.calculatedbathnbr.fillna(value = df['calculatedbathnbr'].mean())

In [12]:
 df['calculatedfinishedsquarefeet'] = df.calculatedfinishedsquarefeet.fillna(value = df['calculatedfinishedsquarefeet'].mean())

In [13]:
 df['finishedsquarefeet12'] = df.finishedsquarefeet12.fillna(value = df['finishedsquarefeet12'].mean())

In [14]:
 df['fullbathcnt'] = df.fullbathcnt.fillna(value = df['fullbathcnt'].mean())

In [15]:
 df['lotsizesquarefeet'] = df.lotsizesquarefeet.fillna(value = df['lotsizesquarefeet'].mean())

In [16]:
df = df.drop(columns=['propertyzoningdesc'])

In [17]:
df['regionidcity'] = df.regionidcity.fillna(value = df['regionidcity'].mode())

In [18]:
df['regionidzip'] = df.regionidzip.fillna(value = df['regionidzip'].mode())

In [19]:
df['unitcnt'] = df.unitcnt.fillna(value = df['unitcnt'].mean())

In [20]:
df['yearbuilt'] = df.yearbuilt.fillna(value = df['yearbuilt'].mode())

In [21]:
df['structuretaxvaluedollarcnt'] = df.structuretaxvaluedollarcnt.fillna(value = df['structuretaxvaluedollarcnt'].mean())

In [22]:
df['censustractandblock'] = df.censustractandblock.fillna(value = df['censustractandblock'].mean())

In [23]:
df = df.drop(columns=['heatingorsystemdesc'])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 1113 to 41622
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      4998 non-null   int64  
 1   propertylandusetypeid         4998 non-null   float64
 2   heatingorsystemtypeid         4998 non-null   float64
 3   id                            4998 non-null   int64  
 4   bathroomcnt                   4998 non-null   float64
 5   bedroomcnt                    4998 non-null   float64
 6   buildingqualitytypeid         4998 non-null   float64
 7   calculatedbathnbr             4998 non-null   float64
 8   calculatedfinishedsquarefeet  4998 non-null   float64
 9   finishedsquarefeet12          4998 non-null   float64
 10  fips                          4998 non-null   float64
 11  fullbathcnt                   4998 non-null   float64
 12  latitude                      4998 non-null   float64
 13 

In [25]:
df = df.drop(columns=['regionidcity', 'regionidcounty', 'regionidzip'])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 1113 to 41622
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      4998 non-null   int64  
 1   propertylandusetypeid         4998 non-null   float64
 2   heatingorsystemtypeid         4998 non-null   float64
 3   id                            4998 non-null   int64  
 4   bathroomcnt                   4998 non-null   float64
 5   bedroomcnt                    4998 non-null   float64
 6   buildingqualitytypeid         4998 non-null   float64
 7   calculatedbathnbr             4998 non-null   float64
 8   calculatedfinishedsquarefeet  4998 non-null   float64
 9   finishedsquarefeet12          4998 non-null   float64
 10  fips                          4998 non-null   float64
 11  fullbathcnt                   4998 non-null   float64
 12  latitude                      4998 non-null   float64
 13 

In [29]:
df['age'] = date.today().year - df.yearbuilt
df = df.drop(columns=['yearbuilt'])

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 1113 to 41622
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      4998 non-null   int64  
 1   propertylandusetypeid         4998 non-null   float64
 2   heatingorsystemtypeid         4998 non-null   float64
 3   id                            4998 non-null   int64  
 4   bathroomcnt                   4998 non-null   float64
 5   bedroomcnt                    4998 non-null   float64
 6   buildingqualitytypeid         4998 non-null   float64
 7   calculatedbathnbr             4998 non-null   float64
 8   calculatedfinishedsquarefeet  4998 non-null   float64
 9   finishedsquarefeet12          4998 non-null   float64
 10  fips                          4998 non-null   float64
 11  fullbathcnt                   4998 non-null   float64
 12  latitude                      4998 non-null   float64
 13 

In [31]:
df['age'] = df.age.fillna(value = df['age'].mean())

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 1113 to 41622
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      4998 non-null   int64  
 1   propertylandusetypeid         4998 non-null   float64
 2   heatingorsystemtypeid         4998 non-null   float64
 3   id                            4998 non-null   int64  
 4   bathroomcnt                   4998 non-null   float64
 5   bedroomcnt                    4998 non-null   float64
 6   buildingqualitytypeid         4998 non-null   float64
 7   calculatedbathnbr             4998 non-null   float64
 8   calculatedfinishedsquarefeet  4998 non-null   float64
 9   finishedsquarefeet12          4998 non-null   float64
 10  fips                          4998 non-null   float64
 11  fullbathcnt                   4998 non-null   float64
 12  latitude                      4998 non-null   float64
 13 

In [38]:
def train_validate_test_split(df):
    train_validate, test = train_test_split(df, test_size=0.2, random_state=123, stratify=df['logerror'])
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=123, stratify=train_validate['logerror'])
    return train, validate, test

In [39]:
df = train_validate_test_split(df)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

1. Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

2. Answer those questions through a mix of statistical tests and visualizations.